In [ ]:
pip install opencv-python opencv-contrib-python  pillow

# CREATE 2 FILES CALLED trainer.yml and names.json and folder called images and remember its paths

# download haarcascade_frontalface_default.xml and remember its path

# CLEAR ALL THE PATHS BEFORE STARTING

In [ ]:
import os
import shutil

# Define the paths to the files and directories
trainer_path = r"C:\Users\hp\Desktop\facial recognition system\trainer.yml"
names_json_path = r"C:\Users\hp\Desktop\facial recognition system\names.json"
images_directory = r"C:\Users\hp\Desktop\facial recognition system\images"  # Directory to clear

def clear_file(file_path: str) -> None:
    """
    Clear the contents of the specified file.

    Parameters:
        file_path (str): The path to the file to be cleared.
    """
    with open(file_path, 'w') as file:
        file.write('')  # Clear the file contents
    print(f'[INFO] {file_path} has been cleared.')

def clear_directory(directory: str) -> None:
    """
    Clear all files and subdirectories in the specified directory.

    Parameters:
        directory (str): The path of the directory to be cleared.
    """
    if not os.path.exists(directory):
        print(f'[ERROR] The directory {directory} does not exist.')
        return

    # Iterate through all files and subdirectories in the directory
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)  # Remove file
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Remove directory and its contents
        except Exception as e:
            print(f'[ERROR] Unable to delete {file_path}. Reason: {e}')
    
    print(f'[INFO] Directory {directory} has been cleared.')

# Clear the trainer file
clear_file(trainer_path)

# Clear the names JSON file with an empty JSON object
with open(names_json_path, 'w') as file:
    file.write('{}')  # Write an empty JSON object
print(f'[INFO] {names_json_path} has been cleared.')

# Clear the images directory
clear_directory(images_directory)



# TRAIN MODEL USING WEBCAM

In [ ]:
import cv2
import numpy as np
from PIL import Image
import os

def create_directory(directory: str) -> None:
    """
    Create a directory if it doesn't exist.

    Parameters:
        directory (str): The path of the directory to be created.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)

def get_face_id(directory: str) -> int:
    """
    Get the first identifier available

    Parameters:
        directory (str): The path of the directory of images.
    """
    user_ids = []
    for filename in os.listdir(directory):
        number = int(os.path.split(filename)[-1].split("-")[1])
        user_ids.append(number)
    user_ids = sorted(list(set(user_ids)))
    max_user_ids = 1 if len(user_ids) == 0 else max(user_ids) + 1
    for i in sorted(range(0, max_user_ids)):
        try:
            if user_ids.index(i):
                face_id = i
        except ValueError as e:
            return i
    return max_user_ids

def save_name(face_id: int, face_name: str, filename: str) -> None:
    """
    Save name on JSON of names

    Parameters:
        face_id (int): The identifier of user.
        face_name (str): The user name.
        filename (str): The file name where to save the pair identifier - name.
    """
    names_json = None
    if os.path.exists(filename):
        with open(filename, 'r') as fs:
            names_json = json.load(fs)
    if names_json is None:
        names_json = {}
    names_json[face_id] = face_name
    with open(filename, 'w') as fs:
        json_dump = json.dumps(names_json, ensure_ascii=False, indent=4)
        fs.write(json_dump)

if __name__ == '__main__':
    directory = r"C:\Users\hp\Desktop\images"
    cascade_classifier_filename = r"C:\Users\hp\Downloads\haarcascade_frontalface_default.xml"
    names_json_filename = 'names.json'

    # Create 'images' directory if it doesn't exist
    create_directory(directory)
    
    # Load the pre-trained face cascade classifier
    faceCascade = cv2.CascadeClassifier(cascade_classifier_filename)
    
    # Open a connection to the default camera (camera index 0)
    cam = cv2.VideoCapture(0)
    
    # Set camera dimensions
    cam.set(3, 640)
    cam.set(4, 480)
    
    # Initialize face capture variables
    count = 0
    face_name = input('\nEnter user name and press <return> -->  ')
    face_id = get_face_id(directory)
    save_name(face_id, face_name, names_json_filename)
    print('\n[INFO] Initializing face capture. Look at the camera and wait...')

    while True:
        # Read a frame from the camera
        ret, img = cam.read()
    
        # Convert the frame to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
        # Detect faces in the frame
        faces = faceCascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    
        # Process each detected face
        for (x, y, w, h) in faces:
            # Draw a rectangle around the detected face
            cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
            # Increment the count for naming the saved images
            count += 1

            # Save the captured image into the 'images' directory
            cv2.imwrite(f'{directory}/Users-{face_id}-{count}.jpg', gray[y:y+h, x:x+w])
    
            # Display the image with rectangles around faces
            cv2.imshow('image', img)
    
        # Press Escape to end the program
        k = cv2.waitKey(100) & 0xff
        if k < 30:
            break
    
        # Take 30 face samples and stop video. You may increase or decrease the number of
        # images. The more, the better while training the model.
        elif count >= 30:
            break
    
    print('\n[INFO] Success! Exiting Program.')
    
    # Release the camera
    cam.release()
    
    # Close all OpenCV windows
    cv2.destroyAllWindows()


# TRAIN THE MODEL USING IMAGE PATH

In [ ]:
import cv2
import os
import json

def create_directory(directory: str) -> None:
    if not os.path.exists(directory):
        os.makedirs(directory)

def get_face_id(directory: str) -> int:
    user_ids = []
    for filename in os.listdir(directory):
        try:
            # Extract number from filenames following the pattern 'Users-{face_id}-{count}.jpg'
            number = int(os.path.split(filename)[-1].split("-")[1])
            user_ids.append(number)
        except (IndexError, ValueError):
            # For files that don't match the expected format, we process them anyway
            continue
    user_ids = sorted(list(set(user_ids)))
    max_user_ids = 1 if len(user_ids) == 0 else max(user_ids) + 1
    return max_user_ids

def save_name(face_id: int, face_name: str, filename: str) -> None:
    names_json = {}
    if os.path.exists(filename):
        with open(filename, 'r') as fs:
            names_json = json.load(fs)
    names_json[face_id] = face_name
    with open(filename, 'w') as fs:
        json.dump(names_json, fs, ensure_ascii=False, indent=4)

def process_image(image_path: str, save_path: str) -> None:
    img = cv2.imread(image_path)
    if img is None:
        print(f'[ERROR] Unable to load image: {image_path}')
        return

    # Convert image to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Resize image to 255x255
    resized_img = cv2.resize(gray_img, (255, 255))

    # Save the processed image
    cv2.imwrite(save_path, resized_img)
    print(f'[INFO] Image saved as {save_path}')

def main():
    biden_directory = r"C:\Users\hp\Desktop\biden"  # Directory with x's person images
    image_directory = r"C:\Users\hp\Desktop\images"  # Directory to save processed images
    names_json_filename = 'names.json'

    create_directory(image_directory)

    face_name = input('\nEnter user name and press <return> -->  ')
    face_id = get_face_id(image_directory)
    save_name(face_id, face_name, names_json_filename)
    print('\n[INFO] Processing images from the Biden directory.')

    # Initialize counter
    count = len([f for f in os.listdir(image_directory) if f.startswith(f'Users-{face_id}-')]) + 1

    for filename in os.listdir(biden_directory):
        image_path = os.path.join(biden_directory, filename)
        
        if not os.path.isfile(image_path):
            continue
        
        new_image_path = os.path.join(image_directory, f'Users-{face_id}-{count}.jpg')
        process_image(image_path, new_image_path)

        # Increment count for the next image
        count += 1

    print('\n[INFO] Success! Exiting Program.')

if __name__ == '__main__':
    main()


# CODE TO TRAIN

In [ ]:
import cv2
import numpy as np
from PIL import Image
import os


if __name__ == "__main__":
    
    # Directory path where the face images are stored.
    path = r"C:\Users\hp\Desktop\images"
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    print("\n[INFO] Training...")
    # Haar cascade file for face detection
    detector = cv2.CascadeClassifier(r"C:\Users\hp\Downloads\haarcascade_frontalface_default.xml")
    
    def getImagesAndLabels(path):
        """
        Load face images and corresponding labels from the given directory path.
    
        Parameters:
            path (str): Directory path containing face images.
    
        Returns:
            list: List of face samples.
            list: List of corresponding labels.
        """
        imagePaths = [os.path.join(path, f) for f in os.listdir(path)]
        faceSamples = []
        ids = []
    
        for imagePath in imagePaths:
            # Convert image to grayscale
            PIL_img = Image.open(imagePath).convert('L')
            img_numpy = np.array(PIL_img, 'uint8')
            
            # Extract the user ID from the image file name
            id = int(os.path.split(imagePath)[-1].split("-")[1])
    
            # Detect faces in the grayscale image
            faces = detector.detectMultiScale(img_numpy)
    
            for (x, y, w, h) in faces:
                # Extract face region and append to the samples
                faceSamples.append(img_numpy[y:y+h, x:x+w])
                ids.append(id)
    
        return faceSamples, ids
    
    faces, ids = getImagesAndLabels(path)
    
    # Train the recognizer with the face samples and corresponding labels
    recognizer.train(faces, np.array(ids))
    
    # Save the trained model into the current directory
    recognizer.write('trainer.yml')
    
    print("\n[INFO] {0} faces trained. Exiting Program".format(len(np.unique(ids))))

# OUTPUT

In [ ]:
import cv2
import numpy as np
import json
import os

if __name__ == "__main__":
    
    # Path to the Haar cascade file for face detection
    face_cascade_path = r"C:\Users\hp\Downloads\haarcascade_frontalface_default.xml"
    
    # Path to the trained model
    trainer_path = r"C:\Users\hp\trainer.yml"
    
    # Path to the names JSON file
    names_json_path = r"C:\Users\hp\names.json"
    
    # Create LBPH Face Recognizer
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    
    # Load the trained model
    if os.path.exists(trainer_path):
        recognizer.read(trainer_path)
    else:
        raise FileNotFoundError(f"Trained model not found at {trainer_path}")

    print(recognizer)
    
    # Create a face cascade classifier
    faceCascade = cv2.CascadeClassifier(face_cascade_path)
    
    # Font for displaying text on the image
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    # Initialize user IDs and associated names
    names = ['None']
    if os.path.exists(names_json_path):
        with open(names_json_path, 'r') as fs:
            names_dict = json.load(fs)
            names = list(names_dict.values())
    else:
        raise FileNotFoundError(f"Names JSON file not found at {names_json_path}")
    
    # Video Capture from the default camera (camera index 0)
    cam = cv2.VideoCapture(0)
    cam.set(3, 640)  # Set width
    cam.set(4, 480)  # Set height
    
    # Minimum width and height for the window size to be recognized as a face
    minW = 0.1 * cam.get(3)
    minH = 0.1 * cam.get(4)
    
    while True:
        # Read a frame from the camera
        ret, img = cam.read()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
        # Detect faces in the frame
        faces = faceCascade.detectMultiScale(
            gray,
            scaleFactor=1.2,
            minNeighbors=5,
            minSize=(int(minW), int(minH)),
        )
    
        for (x, y, w, h) in faces:
            # Recognize the face using the trained model
            id, confidence = recognizer.predict(gray[y:y + h, x:x + w])
            
            # Determine rectangle color and confidence text based on recognition
            if confidence < 51:
                try:
                    # Recognized face
                    name = names[id]
                    confidence_text = f"{round(confidence)+30}%"
                    color = (0, 0, 255)  # Red color for recognized face
                except IndexError:
                    name = " "
                    confidence_text = "N/A"
                    color = (0, 0, 255)  # Red color for recognized face
            else:
                # Unknown face
                name = "Civilian"
                confidence_text = "N/A"
                color = (0, 255, 0)  # Green color for unknown face
    
            # Draw a rectangle around the detected face
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
    
            # Display the recognized name and confidence level on the image
            cv2.putText(img, name, (x + 5, y - 5), font, 1, (255, 255, 255), 2)
            cv2.putText(img, confidence_text, (x + 5, y + h - 5), font, 1, (255, 255, 0), 1)
    
        # Display the image with rectangles around faces
        cv2.imshow('camera', img)
    
        # Press Escape to exit the webcam / program
        k = cv2.waitKey(10) & 0xff
        if k == 27:
            break
    
    print("\n[INFO] Exiting Program.")
    # Release the camera
    cam.release()
    # Close all OpenCV windows
    cv2.destroyAllWindows()

